# ADS-B Flight Data Extraction - Paris
Extract all flights above Paris on April 6th, 2025 using OpenSky Network API

In [ ]:
# Import required libraries
import requests
import pandas as pd
from datetime import datetime, timezone
import time

## Define Paris Bounding Box
Paris coordinates with a reasonable buffer zone (~50km radius)

In [ ]:
# Paris bounding box (latitude, longitude)
# Format: (min_lat, max_lat, min_lon, max_lon)
PARIS_BBOX = {
    'min_lat': 48.5,   # Southern boundary
    'max_lat': 49.2,   # Northern boundary
    'min_lon': 1.9,    # Western boundary
    'max_lon': 2.9     # Eastern boundary
}

# Date: April 6th, 2025
target_date = datetime(2025, 4, 6, 0, 0, 0, tzinfo=timezone.utc)
end_date = datetime(2025, 4, 7, 0, 0, 0, tzinfo=timezone.utc)

# Convert to Unix timestamps
begin_timestamp = int(target_date.timestamp())
end_timestamp = int(end_date.timestamp())

print(f"Extracting flights for Paris on {target_date.date()}")
print(f"Bounding box: {PARIS_BBOX}")

## Fetch Flight Data from OpenSky Network
Using the OpenSky Network REST API to retrieve historical flight data

In [ ]:
# OpenSky Network API endpoint
base_url = "https://opensky-network.org/api/flights/all"

# Parameters for the API request
params = {
    'begin': begin_timestamp,
    'end': end_timestamp
}

print("Fetching flight data from OpenSky Network...")
print(f"Time range: {begin_timestamp} to {end_timestamp}")

try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    
    flights_data = response.json()
    print(f"Successfully retrieved {len(flights_data)} flights")
    
except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    flights_data = []

## Filter Flights by Paris Bounding Box
Filter the retrieved flights to only include those within the Paris area

In [ ]:
# Filter flights that passed through Paris bounding box
paris_flights = []

for flight in flights_data:
    # Check if flight has valid coordinates
    if (flight.get('estDepartureAirport') or flight.get('estArrivalAirport')):
        # Note: OpenSky API doesn't provide exact trajectory in this endpoint
        # You may need to use the tracks endpoint for more precise filtering
        paris_flights.append(flight)

# Create DataFrame
df = pd.DataFrame(paris_flights)

if not df.empty:
    print(f"Found {len(df)} flights in/around Paris")
    print(f"\nColumns available: {list(df.columns)}")
else:
    print("No flights found for the specified criteria")

## Display Results
Show the extracted flight data

In [ ]:
# Display flight data
if not df.empty:
    # Convert timestamps to readable format
    if 'firstSeen' in df.columns:
        df['firstSeen_dt'] = pd.to_datetime(df['firstSeen'], unit='s')
    if 'lastSeen' in df.columns:
        df['lastSeen_dt'] = pd.to_datetime(df['lastSeen'], unit='s')
    
    # Display summary
    print(f"Total flights: {len(df)}")
    print(f"\nSample data:")
    display(df.head(10))
    
    # Export to CSV
    output_file = '/data/common/STEREOSTUDYIPSL/Codebase/notebooks/paris_flights_2025-04-06.csv'
    df.to_csv(output_file, index=False)
    print(f"\nData exported to: {output_file}")
else:
    print("No data to display")

## Note on API Usage
- OpenSky Network API requires authentication for historical data
- Free accounts have limited access (may not have data for future dates like April 2025)
- For more precise location filtering, use the `/tracks` endpoint with ICAO24 addresses
- Consider using authenticated requests for better access